# Extracting Original Characters for movies from IMDB

In [1]:
from difflib import SequenceMatcher
import pandas as pd
import imdb
import requests
import traceback
from nltk import pos_tag, word_tokenize
import nltk
from collections import Counter
import spacy
import pandas as pd
import traceback
import re
import math
spacy_nlp  = spacy.load('en_core_web_sm')

## Getting Orignal Character Names

In [2]:
def getCharactersNames_IMDB():
    Movie_data = pd.read_csv("Movies.csv")
    Character_Df = pd.DataFrame(columns = ['Name', 'Character Names'])
    movie_name = Movie_data['Title']
    originalmovie_name = Movie_data['Original Title']
    original_name = ""
    characterNames = []
    imdbobj = imdb.IMDb()
    for index in range(len(originalmovie_name)):
        try:
            characterNames = []
            original_name = originalmovie_name[index].strip()
            name = movie_name[index].strip()
            print(name)
            movies = imdbobj.search_movie(original_name)
            Code = movies[0].movieID
            movie = imdbobj.get_movie(Code)
            actors = movie['cast']
            for actor in actors:
                if(actor.currentRole and isinstance(actor.currentRole, imdb.utils.RolesList)):
                    for role in actor.currentRole:
                        characterNames.append(getattr(role, 'data')['name'])
                elif(actor.currentRole):
                    characterNames.append(getattr(actor.currentRole, 'data')['name'])
            characterNamesSet = set(characterNames)
            Character_Df = Character_Df.append(pd.Series([name, characterNamesSet],
                                             index = ['Name', 'Character Names']),
                                   ignore_index=True)
        except:
            continue
    Character_Df.to_csv("Original_Characters.csv")


In [3]:
getCharactersNames_IMDB()

Godfather
Shawshank Redemption
Schindler's List
Raging Bull
Casablanca
Citizen Kane
Gone with  Wind
One Flew Over  Cuckoo's Nest
Vertigo
Psycho
Godfather Part II
On the Waterfront
Sunset Blvd
Forrest Gump
12 Angry Men
Star Wars
2001 A Space Odyssey
Chinatown
Singin' in the Rain
It's a Wonderful Life
Some Like It Hot
Ben-Hur
Apocalypse Now
Amadeus
Lord of the Rings The Return of the King
Gladiator
Titanic
From Here to Eternity
Saving Private Ryan
Unforgiven
Rocky
A Streetcar Named Desire
To Kill a Mockingbird
An American in Paris
Best Years of Our Lives
A Clockwork Orange
Doctor Zhivago
Jaws
Patton
Platoon
Braveheart
Dances with Wolves
Jurassic Park
Exorcist
Pianist
Goodfellas
Deer Hunter
All Quiet on  the Western Front
Bonnie and Clyde
French Connection
City Lights
It Happened One Night
A Place in the  Sun
Midnight Cowboy
Mr. Smith Goes to Washington
Rain Man
Annie Hall
Fargo
Giant
Shane
Grapes of Wrath
Green Mile
Close Encounters of the Third Kind
Nashville
Network
Graduate
American G

## Extracting Character from scripts

In [4]:
class NamedEntityExtractor:
    def extract(self, text: str) :
        doc = spacy_nlp(text.strip())
        named_entities = set()

        for i in doc.ents:
            entry = str(i.lemma_).lower()
            text = text.replace(str(i).lower(), "")
            if i.label_ in ["PERSON"]:
                named_entities.add(entry.title())
        named_entities_list = list(named_entities)
        return named_entities_list

def clean_text(text):
    # remove backslash-apostrophe 
    text = re.sub("\'", "", text) 
    # remove everything except alphabets 
    text = re.sub("[^a-zA-Z]"," ",text) 
    # remove whitespaces 
    text = ' '.join(text.split()) 
    # convert text to lowercase 
#     text = text.lower() 
    
    return text

def FinalExtraction():
    named_entity_extractor = NamedEntityExtractor()
    characterNamesFile = pd.read_csv('Original_Characters.csv')
    titles = characterNamesFile['Name']
    knows_titles=[]
    predictedCharacterNames = []
    for title in titles:
        try:
            f = open("scripts/"+title+".txt",'r')
            text = f.read()
            text = clean_text(text)
            named_entities = named_entity_extractor.extract(text)
            named_entities_nouns = []
            for role in named_entities:
                if(role):
                    tagged_role = pos_tag(role.split(";"))
                    [named_entities_nouns.append(word) for word,pos in tagged_role if pos in ("NN", "NNP", "NNS")]
            named_entities_set = set(named_entities_nouns)
            print(f"named entities - {named_entities_set}\n")
            predictedCharacterNames.append(named_entities_set)
            knows_titles.append(title)

        except Exception:
            print(traceback.print_exc())
            continue
    return knows_titles,predictedCharacterNames 


In [6]:
titles,names = FinalExtraction()

named entities - {'Michael Hagen Mama', 'Michael We', 'Mike Michael', 'Ext', 'Right Moe Moe', 'Michael Hurry', 'Michael Enzo', 'Moe Green', 'Woltz', 'Tessio You', 'Michael Ciao', 'Frank', 'Michael Tom Tom', 'Luca Brasis', 'Corleone Ill', 'Speak', 'Luca Brasi', 'Vitelli', 'Tom Michael', 'Palermo Michael', 'Sollozzo', 'Sicilian Michael', 'Michael Fabrizzio', 'Michael Mama Fredo', 'Tessio He', 'Kraut Mick', 'Khartoum', 'Paulie You', 'Michael Innkeeper', 'Dago', 'Carlos', 'Michael Ill', 'Lampone', 'Kay Al Capone', 'Tom', 'Appolonia Michael', 'Mccluskey Michael Lets', 'Michael Michael', 'Mccluskeys', 'Pea', 'Clemenzas', 'Michael Francis Rizzi Flash', 'Tom Tom', 'Michael You', 'Nightclub', 'Mike Mccluskey', 'Sonny Michael', 'Michael Sonny', 'Michael Gunfire', 'Sollozzo Find', 'Moe Buy', 'Rocco Lampone', 'Kay Kay', 'Mccluskey Phil', 'Connie', 'Kay Michael', 'Sonny Talk', 'Mccluskey', 'Strachi Cuneo', 'Michael Tom', 'Moe Greene', 'Roccos', 'Michael Sit', 'Paulie Paulie', 'Tom Hagen', 'Joseph Z

named entities - {'Ilsa Kiss', 'Berger', 'Rick', 'Herr Rick We', 'Rick Ugarte Rick', 'Ho', 'Russian Carl', 'Rick Oh', 'Rick Mademoiselle', 'Lund', 'Yvonne Rick', 'Herr Rick Sacha', 'Rick Do', 'Rick Ill', 'Monsieur Blaine', 'Reich', 'Wacht', 'Ilsa Richard Richard', 'Jan Rick', 'Monsieur Au', 'De Gaulle Can', 'Ilsa Laszlo', 'Louis The', 'Reich Rick', 'Mademoiselle Ilsa', 'Veuve Cliquot', 'Sam Ilsa', 'Richard Rick', 'Ferrari Signor Ferrari', 'Ilsa Play', 'Richard Victor', 'Laszlo', 'Rick Ilsa', 'Louis Ill', 'Rick Put', 'Enjoy', 'Rick Ugarte', 'Gestapo', 'Jan', 'Rick Goodnight Rick', 'Abdul', 'Ilsa Put', 'Ilsa Say', 'Blaine', 'Ilsa Rick', 'Signor Jan', 'Louis', 'Marseilles', 'Marseilles Rick', 'Rick Goodnight', 'Sam Rick', 'Monsieur Rick Monsieur Rick', 'Leuchtag Liebchen', 'Rick Shut', 'Liberte', 'Monsieur Rick', 'Ilsa Captain', 'Yvonne Sacha German', 'Strasser', 'Mrs Leuchtag', 'Rick Carl', 'Cointreaux', 'Richard Blaine American Age', 'Ill', 'Rick Laszlo', 'Louis Renault', 'Rick Signor F

Traceback (most recent call last):
  File "<ipython-input-4-926e6ae6d1a6>", line 34, in FinalExtraction
    f = open("scripts/"+title+".txt",'r')
FileNotFoundError: [Errno 2] No such file or directory: 'scripts/Gone with  Wind.txt'
Traceback (most recent call last):
  File "<ipython-input-4-926e6ae6d1a6>", line 34, in FinalExtraction
    f = open("scripts/"+title+".txt",'r')
FileNotFoundError: [Errno 2] No such file or directory: "scripts/One Flew Over  Cuckoo's Nest.txt"


named entities - {'Jack Scottie', 'Satisfied Pull', 'Fred Hansen', 'Johnny O Scottie', 'Ext Golden Gate Park Day', 'Madeleine Judy', 'Smiles', 'Midge Wood', 'John Ferguson', 'Sequoia Sempervirens', 'Castro House', 'Midge', 'Scottie Grins', 'Madeleine Dissolve', 'Move', 'Madeleine Search', 'Elsters', 'Press Madeleine', 'Hansen', 'Beyond Scottie', 'Judy Push', 'Closeup Madeleine', 'Saunders', 'Scottie', 'Indicates', 'Madeleine He', 'Judy One', 'Judy Barton', 'Judy Vendor', 'Sister Teresa', 'Johnny Scottie', 'Madeleine Ext Market', 'Jack', 'Ill', 'Judy Take', 'Pause Smiles', 'Madeleine Wander', 'Johnny', 'John Or', 'Mozarts', 'Judy Pull', 'Madeleine Ext Graveyard', 'Acrophobia Midge', 'Gavin Elster Midge Gavin Gavin Elster You', 'Madeleine Long Ago Scottie', 'Madeleine Ext', 'Sweet', 'Madeleine Scottie', 'Mozart', 'Judy Frighten', 'Ferguson', 'Scot', 'Scottie Hansen', 'Madeleine Elster', 'Judy Judy', 'Mozart Wolfgang Amadeus', 'Madeleine', 'Gavin Elster', 'William Fogarty', 'Judy Judy Bar

named entities - {'Terry Nice', 'Jimmy', 'Johnny Ill', 'Solari Terry', 'Terry Got', 'Luke Terry', 'Mutt Murphy', 'Sonny Shut', 'Charleys', 'Joseph', 'Johnny Stow', 'Group Shoot Shape', 'Terry Malloy', 'Kayo Nolan', 'Jackie', 'Joey Doyle', 'Barney Specs Sonny Truck', 'Terry Terry Want', 'Edie', 'Terry Other', 'Terry Nolan', 'Jackie Terry', 'Charley Charley', 'Wham Terry', 'Terry Gee Charley', 'Vincent', 'Terry Look', 'Stand', 'Barry Ill', 'Timothy J Nolan Terry Nolan', 'Moose', 'Terry Approach Pier Entrance', 'Joey Pop', 'Group Shot Pop Nolan', 'Terry Get', 'Barney', 'Gerry G', 'Pier Dayfather Barry', 'Johnny Guts', 'Terry Ill', 'Waterf', 'Edie Edie', 'Bye Edie', 'Glover', 'Long Shot', 'Luke Luke', 'Wilson', 'Terry Malloys', 'Pier Entrance', 'Group Shoot Hatch', 'Behind Johnnys', 'Terry Edie', 'Jimmy You', 'G Charley', 'Luke', 'Terry', 'Cousin Johnny', 'Terry Edie Edie Get', 'Mac Johnny', 'Terry Jocko', 'Johnny I', 'Jackie Jackie', 'Truck Sonny', 'Timothy J Nolan', 'Skins Johnny', 'Nola

...

named entities - {'Russell', 'Wolf', 'Vincent How', 'Mother Butch', 'Jimmy', 'Floyd', 'Wurlitzer', 'Vincent Come', 'Int Jimmies Bedroom Morning Jimmies', 'Fonzie', 'Madonna', 'Butch Land', 'Slashes Maynard', 'Mia Wallace', 'Gimps', 'Vincent Mayonnaise', 'Marsellus Butch', 'Vincent Antwan', 'Jules Keep', 'Jules Cmon Yolanda', 'Vincent Quit', 'Choco', 'Bora Bora', 'Merde', 'Takakura Kenstyle', 'Butch Coolidge', 'Vincent Vincent Royale', 'Vincent Cmon', 'Fabby My', 'Zed Without', 'Brett Ya', 'Marvin', 'Amsterdam Vince', 'Dan', 'Ringos', 'Vincent Goddammit', 'Jerry Quarry', 'Honey Bunny Vincent', 'Aunt Ginny', 'Vincent', 'Magnum Jules O S', 'Vincent Chill', 'Vincent Vega Ed Sullivan', 'Asta', 'Zed Pick', 'Marvin The Wolf Slams', 'Mia Mia', 'George Chuvalo', 'Knoxville Next', 'Horror Butch', 'Zed Wanna', 'Jody', 'Marsellus Zed', 'Zed Maynard', 'Jimmie Jimmie', 'Civic Vince', 'Lance I', 'Falls Flat', 'Brett Jules', 'Raven Mccoy', 'Roger', 'John Q Laws', 'Douglas Sirk', 'Jules Examples Vincen

named entities - {'Phil Sheridan', 'Dallass', 'Handcuffs Scars', 'Hatfield Lordsburg Gatewood', 'Ike', 'Lucy Mallory', 'Chris Gatewood', 'Gatewood', 'Jim Ill', 'Gatewood Gatewoods', 'Jerry Bartender', 'Mrs Mallory Lucy', 'Curly Buck', 'Ed', 'Luke Ike Hank', 'Hatfield Curly Peacock', 'Chris Buck', 'Lucy He S', 'Mrs Mallory', 'Yakima', 'Nickname', 'Ringos', 'Mick Howdy Frank', 'Camera', 'Ringfield', 'Chris Chris', 'Bucks', 'Hatfield Hatfield', 'Luke Shotgun', 'Hatfield', 'Dudley Nichols', 'Barney', 'Gatewood Camera', 'Luke Plummer', 'Ben Hecht', 'Bessie Brownie', 'Luke Mis', 'Billys', 'Lucy Mallory Nancy', 'Chris Doc Boone', 'Curly Ill', 'Doc Boone', 'Hatfield Curly Hatfield', 'Billy Billy Kill', 'Hatfield Doc Boone', 'Luke', 'Pan', 'Billy Billys', 'Mrs Gatewood', 'Dallas Blanchard', 'Lucy Peacock', 'Lucy Hatfield', 'Agatha As', 'Buck Hatfield', 'Lucy Lucy', 'Gatewood Gatewood', 'Billy Ringo', 'Hya Buck', 'Chris Si', 'Dallas Hatfield', 'Gatewood Ringo', 'Sam Inside', 'Buck', 'Lee Ferry',

Traceback (most recent call last):
  File "<ipython-input-4-926e6ae6d1a6>", line 34, in FinalExtraction
    f = open("scripts/"+title+".txt",'r')
FileNotFoundError: [Errno 2] No such file or directory: 'scripts/Wuring Heights.txt'


named entities - {'Stare', 'Kristofferson', 'Janis Joplin', 'Black Wizard', 'Howdy Wizard Killer', 'Thomas Wolfe Gods', 'Midday', 'Charles Palantine', 'Bertrand Russell', 'Button', 'Travis Betsy', 'White Cab Co', 'Blow', 'Malcolm X', 'Ext Charles', 'Bets Betsy', 'Tom', 'Mick Jagger Bob Dylan', 'Move', 'Tom Cmon', 'Travis Travis', 'Hotel', 'Tom Tom', 'Tom Betsy', 'Betsys', 'Push', 'Charlie T Wizard', 'Blush', 'Easy Lay', 'Young', 'C U Of Palantine', 'Kris Kristofferson', 'Ask Travis', 'Hubert Humphrey', 'Iris Sport', 'Mccall', 'Palantine', 'Bum', 'Nam', 'Agony', 'Toward', 'Damn Betsy', 'Ouch Betsy', 'Charlie', 'Wizard Wizard', 'Ill', 'Dispatcher', 'Off Duty', 'Travis', 'Wesson', 'Michelangelos', 'St Fuckin Mau Mau', 'Sweet Iris', 'Fill', 'Dough Boy', 'Charles', 'Killer', 'Eddie', 'Walter Cronkite', 'Betsy Charles Palantine', 'Iris Iris', 'Colt', 'Easy Andy', 'Jim', 'Taxi Driver Hero', 'Betsy Betsy', 'Iris', 'Damn', 'Amsterdam The Jungle Travis', 'Andy Andy', 'Betsy', 'Peter Fonda', 'Cha

named entities - {'Harry Lauder', 'Tetrazzini', 'Dad Mom', 'Jack Lets', 'Jerry Take', 'Buck Cohan', 'Mary Its', 'George M Cohan', 'Mary That S', 'Hammerstein', 'Sam Divensky', 'Halt', 'George George', 'Yankee Doodle', 'George Better', 'Charlie Say', 'Fireproofed', 'George M Cohan Oh Georgie', 'Johnny Jones', 'Josie', 'Enjoy', 'George Better Excuse', 'Tony Pastor', 'Llewellyn', 'Swell', 'Charlie Case', 'Sullivan', 'Lottie Collins', 'Albee Coster Beall', 'Dennis', 'Mary George', 'Cohan Coen Cohan George M Cohan', 'Josie Cohan', 'George Not', 'George M S', 'Young', 'Harold', 'Harris', 'Mr Cohan', 'Cohan Never', 'Next Left', 'Bill', 'Cohan', 'Dad', 'Riley', 'Nelson Monument', 'Ill', 'Greek Ill', 'Nellie', 'Hirohito', 'Delmonicos', 'Stix Small', 'Jerry Cohan', 'George Washington', 'Mary You', 'Todd Sloan', 'Mary A', 'George Washington Jr', 'Sam', 'Cigarette', 'Hardships', 'George M Cohan Occupation Actor', 'George M Cohan Print', 'Eddie Foy Ward', 'Mac', 'Vesta', 'Dietz Goff', 'Lewis', 'Har

In [7]:
ExtractedCharacters = pd.DataFrame({'Movie Names':titles,"Character Name":names})

In [8]:
ExtractedCharacters.head()

,Movie Names,Character Name
0,Godfather,"{Michael Hagen Mama, Michael We, Mike Michael,..."
1,Shawshank Redemption,"{Andy Pleasure, Floyd Smokes Put, Wiley, Floyd..."
2,Schindler's List,"{Clerk Good, Lena Schindler, Goeths Adler, Amo..."
3,Raging Bull,"{Jack Jake, Jake Get Everyone Out Of, Jake Ton..."
4,Casablanca,"{Ilsa Kiss, Berger, Rick, Herr Rick We, Rick U..."


In [9]:
df = pd.read_csv('Original_Characters.csv',index_col = 'Unnamed: 0')
df.head()

,Name,Character Names
0,Godfather,"{""Lou - Sollozzo's Driver"", ""Luca Brasi's Assa..."
1,Shawshank Redemption,"{""Andy Dufresne's Wife"", 'Laundry Bob', 'Fat A..."
2,Schindler's List,"{'Majola', 'Gestapo Clerk Klaus Tauber', 'Wilh..."
3,Raging Bull,"{'Heckler in Bar', 'Charlie - Man with Como', ..."
4,Casablanca,"{'Arab Vendor', 'Berger', ""Gambler at Rick's"",..."


In [10]:
ExtractedCharacters.to_csv("Extracted_Characters.csv")

In [11]:
original_df = pd.read_csv("Original_Characters.csv")

In [12]:
original_df.head()

,Unnamed: 0,Name,Character Names
0,0,Godfather,"{""Lou - Sollozzo's Driver"", ""Luca Brasi's Assa..."
1,1,Shawshank Redemption,"{""Andy Dufresne's Wife"", 'Laundry Bob', 'Fat A..."
2,2,Schindler's List,"{'Majola', 'Gestapo Clerk Klaus Tauber', 'Wilh..."
3,3,Raging Bull,"{'Heckler in Bar', 'Charlie - Man with Como', ..."
4,4,Casablanca,"{'Arab Vendor', 'Berger', ""Gambler at Rick's"",..."


In [13]:
ExtractedCharacters.columns=['Name',"Extracted Name"]

In [14]:
ExtractedCharacters.head()

,Name,Extracted Name
0,Godfather,"{Michael Hagen Mama, Michael We, Mike Michael,..."
1,Shawshank Redemption,"{Andy Pleasure, Floyd Smokes Put, Wiley, Floyd..."
2,Schindler's List,"{Clerk Good, Lena Schindler, Goeths Adler, Amo..."
3,Raging Bull,"{Jack Jake, Jake Get Everyone Out Of, Jake Ton..."
4,Casablanca,"{Ilsa Kiss, Berger, Rick, Herr Rick We, Rick U..."


In [15]:
final = pd.merge(left=ExtractedCharacters,right =original_df,on ='Name')

In [16]:
final.to_csv("Assignment2_submission.csv")

### Evaluation

In [21]:
def similarity_checker(text1, text2):
    return SequenceMatcher(None, text1, text2).ratio()

try:
    Submission_File = pd.read_csv('Assignment2_submission.csv')
    Names_Character = Submission_File['Character Names']
    names_extractedlist = Submission_File['Extracted Name']
    identicalmatches_total = 0
    similarmatches_total = 0
    DNMatch_total = 0
    c_extrac_total = 0
    c_total = 0
    i = 0
    while i <len(Names_Character):

        if(isinstance(Names_Character[i], float) and math.isnan(Names_Character[i])):
            continue
        if(isinstance(names_extractedlist[i], float) and math.isnan(names_extractedlist[i])):
            continue
        actualNames_Character = Names_Character[i].replace("{", "")
        actualNames_Character = actualNames_Character.replace("}", "")
        actualNames_Character = actualNames_Character.replace(", '", ",'")
        actualNames_Character = actualNames_Character.replace("'", "")
        actualNames_Character = actualNames_Character.replace(", \"", ",\"")
        actualNames_Character = actualNames_Character.replace("\"", "")
        actualNames_Character = actualNames_Character.split(",")
        extractedNames_Character = names_extractedlist[i].replace("{", "")
        extractedNames_Character = extractedNames_Character.replace("}", "")
        extractedNames_Character = extractedNames_Character.replace(", '", ",'")
        extractedNames_Character = extractedNames_Character.replace("'", "")
        extractedNames_Character = extractedNames_Character.replace(", \"", ",\"")
        extractedNames_Character = extractedNames_Character.replace("\"", "")
        extractedNames_Character = extractedNames_Character.split(",")
        identicalmatch = 0
        DnMatch = 0
        c_extra = 0
        similar_match = 0
        c = 0

        for OG_character in actualNames_Character:
            f = False
            try:
                extractedNames_Character.i(OG_character)
                identicalmatch = identicalmatch + 1
                f = True
                c = c + 1
            except:
                name = Submission_File['Name'][i]
                for extractedCharacter in extractedNames_Character:
                    OG_character = OG_character.strip()
                    OG_character = OG_character.replace("'", "")
                    OG_character = OG_character.replace("\"", "")
                    extractedCharacter = extractedCharacter.strip()
                    extractedCharacter = extractedCharacter.replace("'", "")
                    extractedCharacter = extractedCharacter.replace("\"", "")
                    if(len(extractedCharacter) > 5 and OG_character.find(extractedCharacter) != -1):
                        similar_match = similar_match + 1
                        f = True
                        break
                    elif(extractedCharacter.find(OG_character) != -1):
                        similar_match = similar_match + 1
                        f = True
                        break
                    elif(len(extractedCharacter) > 5 and similarity_checker(OG_character, extractedCharacter) > 0.6):
                        similar_match = similar_match + 1
                        f = True
                        break
                if(not f):
                    DnMatch = DnMatch + 1
                c = c + 1
        c_total = c_total + c
        if(len(extractedNames_Character) >= 1):
            c_extra = len(extractedNames_Character) - (identicalmatch + similar_match + DnMatch)
        c_extrac_total = c_extrac_total + c_extra
        identicalmatches_total = identicalmatches_total + identicalmatch
        similarmatches_total = similarmatches_total + similar_match
        DNMatch_total = DNMatch_total + DnMatch
        i +=1
        
    true_p = identicalmatches_total + similarmatches_total
    false_p = c_extrac_total
    false_n = DNMatch_total
    total_matches = true_p
    f1_score = (true_p / (true_p + 0.5*(false_p + false_n)))
    accuracy = (total_matches/c_total)*100
    print("For F1 score, True Positives are sum of Partial Macthes and Identical Matches, False Positives are which my code considered as characters but are actually not characters and False Negatives are The Characters that did not match from the orignal characters.")
    print("F1 score: ", f1_score)
    print("---------------------------------------------")
    print("For Accuracy, Total Matches are True Positives which divided by the total number of characters")
    print("Accuracy:", accuracy)
        
except:
    traceback.print_exc()
    print("\n")


For F1 score, True Positives are sum of Partial Macthes and Identical Matches, False Positives are which my code considered as characters but are actually not characters and False Negatives are The Characters that did not match from the orignal characters.
F1 score:  0.25801749271137026
---------------------------------------------
For Accuracy, Total Matches are True Positives which divided by the total number of characters
Accuracy: 35.45515245938126
